In [37]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 220)


User input & folder setup

In [38]:
year = input("Enter year (e.g., 2025): ").strip()
quarter = input("Enter quarter (Q1, Q2, Q3, Q4): ").strip().upper()

if quarter not in {"Q1", "Q2", "Q3", "Q4"}:
    raise ValueError("Quarter must be one of: Q1, Q2, Q3, Q4")
if not year.isdigit() or len(year) != 4:
    raise ValueError("Year must be a 4-digit number")

BASE_DIR = os.path.join(
    "C:\\Users",
    os.environ.get("USERNAME"),
    "Documents",
    "Equity"
)

HOME = os.path.join(BASE_DIR, f"{year}_{quarter}")
DATA_DIR = os.path.join(HOME, "data")
os.makedirs(DATA_DIR, exist_ok=True)

print("HOME:", HOME)
print("DATA_DIR:", DATA_DIR)


HOME: C:\Users\MANJANID\Documents\Equity\2025_Q3
DATA_DIR: C:\Users\MANJANID\Documents\Equity\2025_Q3\data


Load data & basic cleaning

In [39]:
grades = pd.read_parquet(os.path.join(DATA_DIR, "grades.parquet"))
cashflows = pd.read_parquet(os.path.join(DATA_DIR, "cashflows.parquet"))

required_cols = [
    "FundID","Adj Strategy","Transaction Date","First Closing Date",
    "Adj Drawdown EUR","Adj Repayment EUR","NAV Adjusted EUR","First Grading-P"
]
missing = [c for c in required_cols if c not in cashflows.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

cash = cashflows.rename(columns={"Adj Strategy": "AdjStrategy"}).copy()
cash["TransactionDate"] = pd.to_datetime(cash["Transaction Date"], errors="coerce")
cash["FirstClosingDate"] = pd.to_datetime(cash["First Closing Date"], errors="coerce")

for c in ["Adj Drawdown EUR","Adj Repayment EUR","NAV Adjusted EUR"]:
    cash[c] = pd.to_numeric(cash[c], errors="coerce").fillna(0.0)

cash = cash.dropna(subset=["FundID","TransactionDate","FirstClosingDate"])
cash["QPeriod"] = cash["TransactionDate"].dt.to_period("Q")

print("Cash rows:", len(cash), "| Funds:", cash["FundID"].nunique())
display(cash.head())


Cash rows: 81793 | Funds: 1129


,VC Fund Name,VC Fund Status,First Commitment Date (per Facility),Vintage Year,Commitment in CCY,Commitment in EUR (hist rate),Year of Transaction Date,Quarter of Transaction Date,IRR,NAV in CCY,NAV in EUR (hist rate),Drawn in CCY,Drawn in EUR (hist rate),Capital Repayment in CCY,Capital Repayment in EUR (hist rate),Revenue Repayment in CCY,Revenue Repayment in EUR (hist rate),Adjusted capital repayment CCY,Adjusted capital repayment EUR,CCY name,Category,FundID,Business Key,Fund Name,Fund Workflow Stage,Vintage Year.1,Investment Strategy Set Up,Strategy,AdjStrategy,Main Sector,Stage Focus,Target Fund Size,Team Location,New Team,PPG Comp & Growth,PPG Innovation,PPG Sustainability,"PPG Soc Imp, Sk & HC",Climate,First Closing Date,Final Closing Date,First Commitment Date (All SoF),Last Commitment Date (All SoF),Planned End Date as per legal documentation,Planned end date with add. years as per legal doc,Termination Date,First Grading-P,Fund Currency,SOF Short Name List,Mandate Short Name List,Transaction Quarter,Transaction Date,Commitment EUR,Commitment CCY,Signed Amount EUR,Signed Amount CCY,Adj Drawdown EUR,Adj Repayment EUR,Adj Drawdown CCY,Adj Repayment CCY,Recallable,NAV EUR,NAV CCY,NAV Adjusted EUR,NAV Adjusted CCY,TransactionDate,FirstClosingDate,QPeriod
0,Global Energy Efficiency and Renewable Energy ...,Signed,2008-11-17,2008-01-01,None,0.0000,2011,2,-0.0280,0.000000,0.000000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,9E02E48EF4F74B0DB14902F85BBB0249,MM-2011-013,Global Energy Efficiency and Renewable Energy ...,Terminated,2008.0,Fund of Funds,Infrastructure,Special Structure/ FoF,Energy Efficiency,Infrastructure,200.0000,Luxembourg,Yes,No,No,No,No,No,2008-11-06,2019-02-06,2008-11-17,2019-02-06,2023-11-06,2025-11-06,None,None,EUR,EIF;Geeref - EU (Trusteeship),EIF;GEEREF - EU Trusteeship,2011-Q2,2011-06-30,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,2011-06-30,2008-11-06,2011Q2
1,Global Energy Efficiency and Renewable Energy ...,Signed,2008-11-17,2008-01-01,None,0.0000,2011,3,-0.0334,0.000000,0.000000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,9E02E48EF4F74B0DB14902F85BBB0249,MM-2011-013,Global Energy Efficiency and Renewable Energy ...,Terminated,2008.0,Fund of Funds,Infrastructure,Special Structure/ FoF,Energy Efficiency,Infrastructure,200.0000,Luxembourg,Yes,No,No,No,No,No,2008-11-06,2019-02-06,2008-11-17,2019-02-06,2023-11-06,2025-11-06,None,None,EUR,EIF;Geeref - EU (Trusteeship),EIF;GEEREF - EU Trusteeship,2011-Q3,2011-09-30,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,2011-09-30,2008-11-06,2011Q3
2,Global Energy Efficiency and Renewable Energy ...,Signed,2008-11-17,2008-01-01,None,0.0000,2011,4,-0.0207,0.000000,0.000000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,9E02E48EF4F74B0DB14902F85BBB0249,MM-2011-013,Global Energy Efficiency and Renewable Energy ...,Terminated,2008.0,Fund of Funds,Infrastructure,Special Structure/ FoF,Energy Efficiency,Infrastructure,200.0000,Luxembourg,Yes,No,No,No,No,No,2008-11-06,2019-02-06,2008-11-17,2019-02-06,2023-11-06,2025-11-06,None,None,EUR,EIF;Geeref - EU (Trusteeship),EIF;GEEREF - EU Trusteeship,2011-Q4,2011-12-31,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,2011-12-31,2008-11-06,2011Q4
3,Global Energy Efficiency and Renewable Energy ...,Signed,2008-11-17,2008-01-01,None,0.0000,2011,4,NaN,None,0.000000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,9E02E48EF4F74B0DB14902F85BBB0249,MM-2011-013,Global Energy Efficiency and Renewable Energy ...,Terminated,2008.0,Fund of Funds,Infrastructure,Special Structure/ FoF,Energy Efficiency,Infrastructure,200.0000,Luxembourg,Yes,No,No,No,No,No,2008-11-06,2019-02-06,2008-11-17,2019-02-06,2023-11-06,2025-11-06,None,None,EUR,EIF;Geeref - EU (Trusteeship),EIF;GEEREF - EU Trusteeship,2011-Q4,2011-10-25,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,2011-10-25,2008-11-06,2011Q4
4,Global En

Strategy definitions

In [40]:
DEBT_STRATEGIES = {"Hybrid Debt-Equity", "Private Debt", "Other Private Debt"}
VC_STRATEGY = "Venture Capital"

fund_strategy = (
    cash.groupby("FundID")["AdjStrategy"]
        .agg(lambda s: s.mode().iat[0] if len(s.mode()) else s.iloc[0])
        .reset_index()
)


Quarterly snapshot + DPI / TVPI

In [41]:
q_snap = (
    cash.sort_values(["FundID","TransactionDate"])
        .groupby(["FundID","QPeriod"], as_index=False)
        .agg(
            AdjStrategy=("AdjStrategy","last"),
            FirstClosingDate=("FirstClosingDate","last"),
            QuarterDrawdown=("Adj Drawdown EUR","sum"),
            QuarterRepayment=("Adj Repayment EUR","sum"),
            QuarterEndNAV=("NAV Adjusted EUR","last"),
            QuarterEndDate=("TransactionDate","max"),
        )
)

q_snap = q_snap.sort_values(["FundID","QPeriod"])
q_snap["QuarterEndNAV"] = q_snap.groupby("FundID")["QuarterEndNAV"].ffill().fillna(0)

q_snap["CumDrawdown"] = q_snap.groupby("FundID")["QuarterDrawdown"].cumsum()
q_snap["CumRepayment"] = q_snap.groupby("FundID")["QuarterRepayment"].cumsum()

q_snap["PaidIn"] = q_snap["CumDrawdown"].abs()
q_snap["Distributed"] = q_snap["CumRepayment"].abs()
q_snap["NAV"] = q_snap["QuarterEndNAV"].abs()

q_snap["DPI"] = np.where(q_snap["PaidIn"]>0, q_snap["Distributed"]/q_snap["PaidIn"], np.nan)
q_snap["TVPI"] = np.where(q_snap["PaidIn"]>0, (q_snap["Distributed"]+q_snap["NAV"])/q_snap["PaidIn"], np.nan)

display(q_snap.head())


,FundID,QPeriod,AdjStrategy,FirstClosingDate,QuarterDrawdown,QuarterRepayment,QuarterEndNAV,QuarterEndDate,CumDrawdown,CumRepayment,PaidIn,Distributed,NAV,DPI,TVPI
0,0084126E5AB84A059EFCD1CB88947783,2013Q4,Venture Capital,2013-12-20,0.00,0.0,0.00,2013-12-20,0.00,0.0,0.00,0.0,0.00,NaN,NaN
1,0084126E5AB84A059EFCD1CB88947783,2014Q1,Venture Capital,2013-12-20,11089.78,0.0,8162.25,2014-03-31,11089.78,0.0,11089.78,0.0,8162.25,0.0,0.736016
2,0084126E5AB84A059EFCD1CB88947783,2014Q2,Venture Capital,2013-12-20,0.00,0.0,5166.24,2014-06-30,11089.78,0.0,11089.78,0.0,5166.24,0.0,0.465856
3,0084126E5AB84A059EFCD1CB88947783,2014Q3,Venture Capital,2013-12-20,13499.92,0.0,14561.30,2014-09-30,24589.70,0.0,24589.70,0.0,14561.30,0.0,0.592171
4,0084126E5AB84A059EFCD1CB88947783,2014Q4,Venture Capital,2013-12-20,6575.30,0.0,18893.36,2014-12-31,31165.00,0.0,31165.00,0.0,18893.36,0.0,0.606236


Transaction-level IRR (XIRR)

In [42]:
def xnpv(rate, cfs, dts):
    """
    rate: float
    cfs:  1D numpy array of floats
    dts:  1D numpy array of numpy datetime64[ns]
    """
    dts = np.asarray(dts, dtype="datetime64[ns]")
    cfs = np.asarray(cfs, dtype=float)

    t0 = dts[0]
    # Convert to day counts using numpy, not pandas
    day_counts = (dts - t0) / np.timedelta64(1, "D")
    years = day_counts / 365.0

    return np.sum(cfs / ((1.0 + rate) ** years))


def xirr_newton(cfs, dts, guess=0.1, max_iter=80, tol=1e-7):
    """
    Newton solve for XIRR. Returns np.nan if it fails.
    """
    dts = np.asarray(dts, dtype="datetime64[ns]")
    cfs = np.asarray(cfs, dtype=float)

    rate = float(guess)
    for _ in range(max_iter):
        f = xnpv(rate, cfs, dts)
        if not np.isfinite(f):
            return np.nan
        if abs(f) < tol:
            return rate

        eps = 1e-6
        f1 = xnpv(rate + eps, cfs, dts)
        df = (f1 - f) / eps
        if df == 0 or not np.isfinite(df):
            return np.nan

        rate_new = rate - f / df
        if rate_new <= -0.999999 or not np.isfinite(rate_new):
            return np.nan
        rate = rate_new

    return np.nan


Compute IRR per fund-quarter

In [43]:
def compute_fund_quarter_xirr(fund_cash, fund_q):
    """
    fund_cash: transaction-level rows for one fund
    fund_q:    q_snap rows for same fund (quarters)
    Returns DF with IRR and IRR_Flag aligned to fund_q rows.
    """
    fund_cash = fund_cash.sort_values("TransactionDate")

    # Build transaction-level cashflows (drawdown negative, repayment positive)
    cfs = (-fund_cash["Adj Drawdown EUR"].abs() + fund_cash["Adj Repayment EUR"].abs()).to_numpy(dtype=float)
    dts = fund_cash["TransactionDate"].to_numpy(dtype="datetime64[ns]")

    irr_vals = []
    irr_flags = []

    j = 0
    n = len(cfs)

    for r in fund_q.itertuples(index=False):
        q_end = np.datetime64(r.QuarterEndDate, "ns")

        # include transactions up to quarter end
        while j < n and dts[j] <= q_end:
            j += 1

        cfs_slice = cfs[:j]
        dts_slice = dts[:j]

        # No transactions yet
        if len(cfs_slice) == 0:
            irr_vals.append(np.nan)
            irr_flags.append("no_txns")
            continue

        terminal_nav = float(abs(r.NAV)) if np.isfinite(r.NAV) else 0.0

        cfs_full = np.append(cfs_slice, terminal_nav)
        dts_full = np.append(dts_slice, q_end).astype("datetime64[ns]")

        # Require sign change
        if not (np.any(cfs_full < 0) and np.any(cfs_full > 0)):
            irr_vals.append(np.nan)
            irr_flags.append("no_sign_change")
            continue

        tvpi = r.TVPI
        guess = 0.10 if (pd.notna(tvpi) and tvpi > 1.0) else -0.10

        irr = xirr_newton(cfs_full, dts_full, guess=guess)

        # Flat-TVPI handling (Excel logic)
        if pd.notna(tvpi) and (0.98 <= tvpi <= 1.02):
            if not np.isfinite(irr):
                irr2 = xirr_newton(cfs_full, dts_full, guess=-guess)
                if np.isfinite(irr2):
                    irr = irr2
                    irr_flags.append("flat_retry_success")
                else:
                    irr = 0.0
                    irr_flags.append("flat_forced_0")
            else:
                irr_flags.append("ok_flat")
        else:
            irr_flags.append("ok" if np.isfinite(irr) else "fail")

        irr_vals.append(float(irr) if np.isfinite(irr) else np.nan)

    return pd.DataFrame({"IRR": irr_vals, "IRR_Flag": irr_flags})


Attach IRR to q_snap

In [44]:
irr_rows = []

for fund_id, fund_q in q_snap.groupby("FundID", sort=False):
    fund_cash = cash[cash["FundID"] == fund_id]
    irr_df = compute_fund_quarter_xirr(fund_cash, fund_q)

    irr_df = irr_df.copy()
    irr_df["FundID"] = fund_id
    irr_df["QPeriod"] = fund_q["QPeriod"].values

    irr_rows.append(irr_df)

irr_all = pd.concat(irr_rows, ignore_index=True)

q_snap = q_snap.merge(irr_all, on=["FundID", "QPeriod"], how="left")

display(q_snap[["FundID","QPeriod","IRR","IRR_Flag"]].head(20))
display(q_snap["IRR_Flag"].value_counts(dropna=False).to_frame("count"))


,FundID,QPeriod,IRR,IRR_Flag
0,0084126E5AB84A059EFCD1CB88947783,2013Q4,NaN,no_sign_change
1,0084126E5AB84A059EFCD1CB88947783,2014Q1,NaN,fail
2,0084126E5AB84A059EFCD1CB88947783,2014Q2,NaN,fail
3,0084126E5AB84A059EFCD1CB88947783,2014Q3,NaN,fail
4,0084126E5AB84A059EFCD1CB88947783,2014Q4,NaN,fail
5,0084126E5AB84A059EFCD1CB88947783,2015Q1,-0.460517,ok
6,0084126E5AB84A059EFCD1CB88947783,2015Q2,-0.389536,ok
7,0084126E5AB84A059EFCD1CB88947783,2015Q3,-0.298801,ok
8,0084126E5AB84A059EFCD1CB88947783,2015Q4,-0.209064,ok
9,0084126E5AB84A059EFCD1CB88947783,2016Q1,-0.163269,ok


,count
IRR_Flag,
ok,26508
fail,2751
no_sign_change,2647
ok_flat,1466
flat_forced_0,62
flat_retry_success,26


Grade calculation (A/B/C/D)

In [45]:
GRADE_ORDER=["A","B","C","D"]

def quartile_to_grade(s):
    r=s.rank(pct=True)
    return pd.cut(r,[0,.25,.5,.75,1],labels=["D","C","B","A"],include_lowest=True)

q_snap["Grade_DPI"]  = q_snap.groupby(["AdjStrategy","QPeriod"])["DPI"].transform(quartile_to_grade)
q_snap["Grade_TVPI"] = q_snap.groupby(["AdjStrategy","QPeriod"])["TVPI"].transform(quartile_to_grade)
q_snap["Grade_IRR"]  = q_snap.groupby(["AdjStrategy","QPeriod"])["IRR"].transform(quartile_to_grade)


Apply grading rules

In [46]:
# ============================================================
# CELL 10 — Build flags + apply grading rules (A/B/C/D)
# ============================================================

# --- 1) Build fund timing (5+1 if repayments start within 5y else 6+1) ---
rep = cash[cash["Adj Repayment EUR"].abs() > 0].copy()
first_repay = rep.groupby("FundID")["TransactionDate"].min().reset_index(name="FirstRepaymentDate")

first_close = cash.groupby("FundID")["FirstClosingDate"].min().reset_index(name="FirstCloseDate")

fund_timing = first_close.merge(first_repay, on="FundID", how="left")
fund_timing = fund_timing.merge(fund_strategy, on="FundID", how="left")

fund_timing["RepayWithin5Y"] = (
    fund_timing["FirstRepaymentDate"].notna() &
    (fund_timing["FirstRepaymentDate"] <= (fund_timing["FirstCloseDate"] + pd.DateOffset(years=5)))
)

fund_timing["BaseYears"] = np.where(fund_timing["RepayWithin5Y"], 5, 6)
fund_timing["InvestPeriodYears"] = fund_timing["BaseYears"] + 1

# --- 2) Attach timing to q_snap and compute IsInvestmentPeriod ---
q_snap = q_snap.merge(
    fund_timing[["FundID", "FirstCloseDate", "InvestPeriodYears", "AdjStrategy"]],
    on=["FundID", "AdjStrategy"],
    how="left"
)

q_snap["IsInvestmentPeriod"] = q_snap["QuarterEndDate"] <= (
    q_snap["FirstCloseDate"] + q_snap["InvestPeriodYears"].apply(lambda y: pd.DateOffset(years=int(y)))
)

# --- 3) Strategy fund count per quarter (needed for <30 rule) ---
fund_counts = (
    q_snap.groupby(["AdjStrategy", "QPeriod"])["FundID"]
         .nunique()
         .rename("StrategyFundCount")
         .reset_index()
)
q_snap = q_snap.merge(fund_counts, on=["AdjStrategy", "QPeriod"], how="left")

# --- 4) Debt/VC flags ---
q_snap["IsDebt"] = q_snap["AdjStrategy"].isin(DEBT_STRATEGIES)
q_snap["IsVC"] = q_snap["AdjStrategy"].eq(VC_STRATEGY)

# --- 5) Grade helper functions ---
GRADE_ORDER = ["A","B","C","D"]
grade_to_idx = {g:i for i,g in enumerate(GRADE_ORDER)}
idx_to_grade = {i:g for g,i in grade_to_idx.items()}

def worse_grade(g1, g2):
    if pd.isna(g1): return g2
    if pd.isna(g2): return g1
    return g1 if grade_to_idx[g1] >= grade_to_idx[g2] else g2

def downgrade_one_notch(g):
    if pd.isna(g): return g
    return idx_to_grade[min(grade_to_idx[g] + 1, 3)]

# --- 6) Apply the grading logic you specified ---
def final_grade(row):
    # If < 30 funds in strategy-quarter: take lower quartile between DPI and TVPI
    if row["StrategyFundCount"] < 30:
        return worse_grade(row["Grade_DPI"], row["Grade_TVPI"])

    # Debt funds: IRR
    if row["IsDebt"]:
        return row["Grade_IRR"]

    # Equity, investment period: DPI (except VC => TVPI)
    if row["IsInvestmentPeriod"]:
        return row["Grade_TVPI"] if row["IsVC"] else row["Grade_DPI"]

    # Equity, harvesting: IRR primary + DPI complementary (downgrade max 1 notch)
    base = row["Grade_IRR"]
    dpi_g = row["Grade_DPI"]
    if pd.isna(base):
        return base

    if pd.notna(dpi_g) and (grade_to_idx[dpi_g] > grade_to_idx[base]):
        return downgrade_one_notch(base)

    return base

q_snap["CurrentGrade"] = q_snap.apply(final_grade, axis=1)

display(q_snap[["FundID","AdjStrategy","QPeriod","DPI","TVPI","IRR","StrategyFundCount","IsInvestmentPeriod","CurrentGrade"]].head(20))
display(q_snap["CurrentGrade"].value_counts(dropna=False).to_frame("count"))


C:\Users\manjanid\AppData\Local\Temp\ipykernel_7232\1427749335.py:30: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  q_snap["FirstCloseDate"] + q_snap["InvestPeriodYears"].apply(lambda y: pd.DateOffset(years=int(y)))


,FundID,AdjStrategy,QPeriod,DPI,TVPI,IRR,StrategyFundCount,IsInvestmentPeriod,CurrentGrade
0,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2013Q4,NaN,NaN,NaN,107,True,NaN
1,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2014Q1,0.000000,0.736016,NaN,87,True,C
2,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2014Q2,0.000000,0.465856,NaN,103,True,D
3,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2014Q3,0.000000,0.592171,NaN,107,True,D
4,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2014Q4,0.000000,0.606236,NaN,117,True,D
5,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2015Q1,0.000000,0.681545,-0.460517,113,True,D
6,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2015Q2,0.000000,0.796352,-0.389536,124,True,D
7,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2015Q3,0.000000,0.813625,-0.298801,124,True,D
8,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2015Q4,0.000000,0.872087,-0.209064,145,True,C
9,0084126E5AB84A059EFCD1CB88947783,Venture Capital,2016Q1,0.000000,0.860950,-0.163269,135,True,C


,count
CurrentGrade,
C,9267
D,8847
B,8025
A,4685
NaN,2636


Add grades to cashflows (First P-Grade + 4Q blocks)

In [47]:
cash2=cash.copy()
cash2["First Grading-P"]=cashflows["First Grading-P"]

valid_funds=cash2.groupby("FundID")["First Grading-P"].apply(lambda s:s.notna().any())
cash2=cash2[cash2["FundID"].isin(valid_funds[valid_funds].index)]

fund_quarters=cash2[["FundID","QPeriod"]].drop_duplicates().sort_values(["FundID","QPeriod"])
fund_quarters["RankQ"]=fund_quarters.groupby("FundID").cumcount()+1
fund_quarters["Block4"]=(fund_quarters["RankQ"]-1)//4+1

fund_first=cash2.dropna(subset=["First Grading-P"]).groupby("FundID").first()[["First Grading-P"]]
fund_quarters=fund_quarters.merge(fund_first,on="FundID",how="left")
fund_quarters=fund_quarters.merge(q_snap[["FundID","QPeriod","CurrentGrade"]],on=["FundID","QPeriod"],how="left")

fund_quarters["AssignedGrade"]=np.where(
    fund_quarters["Block4"]==1,
    fund_quarters["First Grading-P"],
    fund_quarters["CurrentGrade"]
)

fund_quarters["AssignedGrade"]=fund_quarters.groupby("FundID")["AssignedGrade"].ffill()

cash_with_grade=cash2.merge(
    fund_quarters[["FundID","QPeriod","AssignedGrade"]],
    on=["FundID","QPeriod"],how="left"
).rename(columns={"AssignedGrade":"CurrentGrade"})

display(cash_with_grade.head())

# Save fund_quarters for downstream NAV notebooks
fund_quarters.to_parquet(os.path.join(DATA_DIR, "fund_quarters.parquet"), index=False)
# Optional CSV fallback
# fund_quarters.to_csv(os.path.join(DATA_DIR, "fund_quarters.csv"), index=False)

# Save grade transitions for inspection (quarter-to-quarter)
fq_trans = fund_quarters.sort_values(["FundID", "RankQ"]).copy()
fq_trans["NextGrade"] = fq_trans.groupby("FundID")["AssignedGrade"].shift(-1)
grade_transitions = fq_trans.dropna(subset=["AssignedGrade", "NextGrade"]).copy()
grade_transitions.to_csv(os.path.join(DATA_DIR, "grade_transitions.csv"), index=False)

print("DATA_DIR:", DATA_DIR)
import os
print("Exists:", os.path.exists(os.path.join(DATA_DIR, "fund_quarters.parquet")))



,VC Fund Name,VC Fund Status,First Commitment Date (per Facility),Vintage Year,Commitment in CCY,Commitment in EUR (hist rate),Year of Transaction Date,Quarter of Transaction Date,IRR,NAV in CCY,NAV in EUR (hist rate),Drawn in CCY,Drawn in EUR (hist rate),Capital Repayment in CCY,Capital Repayment in EUR (hist rate),Revenue Repayment in CCY,Revenue Repayment in EUR (hist rate),Adjusted capital repayment CCY,Adjusted capital repayment EUR,CCY name,Category,FundID,Business Key,Fund Name,Fund Workflow Stage,Vintage Year.1,Investment Strategy Set Up,Strategy,AdjStrategy,Main Sector,Stage Focus,Target Fund Size,Team Location,New Team,PPG Comp & Growth,PPG Innovation,PPG Sustainability,"PPG Soc Imp, Sk & HC",Climate,First Closing Date,Final Closing Date,First Commitment Date (All SoF),Last Commitment Date (All SoF),Planned End Date as per legal documentation,Planned end date with add. years as per legal doc,Termination Date,First Grading-P,Fund Currency,SOF Short Name List,Mandate Short Name List,Transaction Quarter,Transaction Date,Commitment EUR,Commitment CCY,Signed Amount EUR,Signed Amount CCY,Adj Drawdown EUR,Adj Repayment EUR,Adj Drawdown CCY,Adj Repayment CCY,Recallable,NAV EUR,NAV CCY,NAV Adjusted EUR,NAV Adjusted CCY,TransactionDate,FirstClosingDate,QPeriod,CurrentGrade
0,Istanbul Venture Capital Initiative iVCi,Signed,2007-11-13,2007-01-01,None,0.0000,2006,4,NaN,None,0.000000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,F9FBAC9D42E54B4AB5D545F9FD4F926D,MM-2012-061,Istanbul Venture Capital Initiative iVCi,Terminated,2007.0,Fund of Funds,Generalist,Special Structure/ FoF,Generalist,Generalist Fund,200000000.0000,Türkiye,No,No,No,No,No,No,2007-11-13,2009-03-31,2007-11-13,2009-02-27,2022-11-13,2022-11-13,None,C,EUR,EIF;RCM7,EIF;RCR,2006-Q4,2006-12-11,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.00,0.00,2006-12-11,2007-11-13,2006Q4,C
1,Istanbul Venture Capital Initiative iVCi,Signed,2007-11-13,2007-01-01,8000000.1000,8000000.1000,2007,4,NaN,None,0.000000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,F9FBAC9D42E54B4AB5D545F9FD4F926D,MM-2012-061,Istanbul Venture Capital Initiative iVCi,Terminated,2007.0,Fund of Funds,Generalist,Special Structure/ FoF,Generalist,Generalist Fund,200000000.0000,Türkiye,No,No,No,No,No,No,2007-11-13,2009-03-31,2007-11-13,2009-02-27,2022-11-13,2022-11-13,None,C,EUR,EIF;RCM7,EIF;RCR,2007-Q4,2007-11-13,8000000.1000,8000000.1000,8000000.1000,8000000.1000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.00,0.00,2007-11-13,2007-11-13,2007Q4,C
2,Istanbul Venture Capital Initiative iVCi,Signed,2007-11-13,2007-01-01,None,0.0000,2007,4,NaN,None,0.000000,4960.0000,4960.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,F9FBAC9D42E54B4AB5D545F9FD4F926D,MM-2012-061,Istanbul Venture Capital Initiative iVCi,Terminated,2007.0,Fund of Funds,Generalist,Special Structure/ FoF,Generalist,Generalist Fund,200000000.0000,Türkiye,No,No,No,No,No,No,2007-11-13,2009-03-31,2007-11-13,2009-02-27,2022-11-13,2022-11-13,None,C,EUR,EIF;RCM7,EIF;RCR,2007-Q4,2007-11-14,0.0000,0.0000,0.0000,0.0000,4960.0,0.0,4960.0,0.0,0.0000,0.0000,0.0000,0.00,0.00,2007-11-14,2007-11-13,2007Q4,C
3,Istanbul Venture Capital Initiative iVCi,Signed,2007-11-13,2007-01-01,None,0.0000,2008,4,NaN,None,0.000000,235040.0000,235040.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,F9FBAC9D42E54B4AB5D545F9FD4F926D,MM-2012-061,Istanbul Venture Capital Initiative iVCi,Terminated,2007.0,Fund of Funds,Generalist,Special Structure/ FoF,Generalist,Generalist Fund,200000000.0000,Türkiye,No,No,No,No,No,No,2007-11-13,2009-03-31,2007-11-13,2009-02-27,2022-11-13,2022-11-13,None,C,EUR,EIF;RCM7,EIF;RCR,2008-Q4,2008-11-11,0.0000,0.0000,0.0000,0.0000,235040.0,0.0,235040.0,0.0,0.0000,139718.1900,139718.1900,139718.19,139718.19,2008-11-11,2007-11-13,2008Q4,C
4,Istanbul Venture Capital Initiative iVCi,Signed,2007-11-13,2007-01-01,None,0.0000,2008,4,-0.9798,139718.190000,139718.190000,None,0.0000,None,0.0000,None,0.0000,None,0.0000,EUR,None,F9FBAC9D

DATA_DIR: C:\Users\MANJANID\Documents\Equity\2025_Q3\data
Exists: False


10-year transition matrices (ONE transition per fund)

In [48]:
# ============================================================
# CELL 12 — 10y transition matrices (ONE transition per fund, Q1 -> Q40)
#           Only Venture Capital + Private Equity
#           Only From in {A,B,C}
#           (No TransactionDate required)
# ============================================================

GRADE_FROM = ["A", "B", "C"]
GRADE_TO   = ["A", "B", "C", "D"]
TARGET_STRATEGIES = {"Venture Capital", "Private Equity"}
TEN_YEARS_QUARTERS = 40

def counts_and_probs(df, from_col="From", to_col="To"):
    counts = pd.crosstab(df[from_col], df[to_col]).reindex(
        index=GRADE_FROM, columns=GRADE_TO, fill_value=0
    )
    probs = counts.div(counts.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
    return counts, probs

# fund_quarters grade column name
GRADE_COL = "AssignedGrade" if "AssignedGrade" in fund_quarters.columns else "QuarterAssignedGrade"
if GRADE_COL not in fund_quarters.columns:
    raise ValueError(f"fund_quarters must contain 'AssignedGrade' or 'QuarterAssignedGrade'. Found: {fund_quarters.columns.tolist()}")

# --- Eligibility: >= 40 ranked quarters (no contiguity) ---
max_rank = fund_quarters.groupby("FundID")["RankQ"].max()
eligible_funds = max_rank[max_rank >= TEN_YEARS_QUARTERS].index
print("Eligible funds (>=40 quarters):", len(eligible_funds))

fq40 = fund_quarters[fund_quarters["FundID"].isin(eligible_funds)].copy()

# --- Fund-level From grade: First Grading-P (first non-null per fund, no date needed) ---
cf_first = cash2[["FundID", "AdjStrategy", "First Grading-P"]].copy()
cf_first["First Grading-P"] = cf_first["First Grading-P"].astype(str).str.strip()
cf_first.loc[cf_first["First Grading-P"].isin(["", "nan", "None", "<NA>"]), "First Grading-P"] = np.nan

fund_from = (
    cf_first.dropna(subset=["First Grading-P"])
            .groupby("FundID", as_index=False)
            .first()[["FundID", "AdjStrategy", "First Grading-P"]]
            .rename(columns={"First Grading-P": "From"})
)

# --- Fund-level To grade: grade at RankQ = 40 from fund_quarters ---
fund_to = (
    fq40[fq40["RankQ"] == TEN_YEARS_QUARTERS][["FundID", GRADE_COL]]
        .rename(columns={GRADE_COL: "To"})
)

# --- Build transitions ---
trans_10y = (
    fund_from.merge(fund_to, on="FundID", how="inner")
             .dropna(subset=["From", "To", "AdjStrategy"])
)

# Filter strategies
trans_10y = trans_10y[trans_10y["AdjStrategy"].isin(TARGET_STRATEGIES)].copy()

# Filter first grade to A/B/C only
trans_10y = trans_10y[trans_10y["From"].isin(GRADE_FROM)].copy()

print("Transitions after filters:", len(trans_10y))
display(trans_10y.head(10))

# --- Combined matrix (VC + PE) ---
combined_counts, combined_probs = counts_and_probs(trans_10y, "From", "To")
print(" COMBINED (VC + PE) Q1->Q40 COUNTS (From=A/B/C):")
display(combined_counts)
print(" COMBINED (VC + PE) Q1->Q40 PROBABILITIES (From=A/B/C):")
display(combined_probs)

# --- Separate matrices ---
for strat in sorted(TARGET_STRATEGIES):
    df_s = trans_10y[trans_10y["AdjStrategy"] == strat]
    c, p = counts_and_probs(df_s, "From", "To")
    print(" ======================")
    print(f"Strategy: {strat} | funds: {df_s['FundID'].nunique()} | transitions: {len(df_s)}")
    print("======================")
    print("Counts:")
    display(c)
    print("Probabilities:")
    display(p)
 
# --- Derive 1-year matrix from the 10-year matrix (VC + PE combined) --- 
STATES = ['A','B','C','D'] 
P10 = combined_probs.reindex(index=GRADE_FROM, columns=GRADE_TO).to_numpy(dtype=float) 

# Build full 4x4 10y matrix; use D as absorbing for the missing row 
P10_full = np.zeros((4,4), dtype=float) 
P10_full[:3, :] = P10 
P10_full[3, 3] = 1.0 

def _matrix_root_via_eig(P, power=10):
    vals, vecs = np.linalg.eig(P)
    vals = vals.astype(complex)
    vals = np.where(np.isfinite(vals), vals, 0.0)
    vals_root = np.emath.power(vals, 1.0 / power)
    P_root = vecs @ np.diag(vals_root) @ np.linalg.inv(vecs)
    P_root = np.real_if_close(P_root, tol=1e-6)
    return np.real(P_root)

 
def _project_row_stochastic(P): 
    P = np.where(np.isfinite(P), P, 0.0) 
    P = np.clip(P, 0.0, None) 
    row_sums = P.sum(axis=1, keepdims=True) 
    row_sums = np.where(row_sums == 0, 1.0, row_sums) 
    return P / row_sums 
 
P1_raw = _matrix_root_via_eig(P10_full, power=10) 
P1 = _project_row_stochastic(P1_raw) 
 
# Implied 10y from the calibrated 1y 
P10_implied = np.linalg.matrix_power(P1, 10) 
max_err = np.abs(P10_implied - P10_full).max() 
print('Max abs error vs observed 10y (after stochastic projection):', round(float(max_err), 8)) 
 
P1_df = pd.DataFrame(P1, index=STATES, columns=STATES) 
P10_implied_df = pd.DataFrame(P10_implied, index=STATES, columns=STATES) 
 
print(' Derived 1y transition matrix (A/B/C/D):') 
display(P1_df) 

# Save combined 1y transition matrix
P1_df.to_csv(os.path.join(DATA_DIR, "grade_transition_1y_all.csv"))

# Build + save strategy-specific 1y matrices (PE, VC)
for strat in sorted(TARGET_STRATEGIES):
    df_s = trans_10y[trans_10y["AdjStrategy"] == strat]
    c_s, p_s = counts_and_probs(df_s, "From", "To")
    P10_s = p_s.reindex(index=GRADE_FROM, columns=GRADE_TO).to_numpy(dtype=float)
    P10_s_full = np.zeros((4,4), dtype=float)
    P10_s_full[:3, :] = P10_s
    P10_s_full[3, 3] = 1.0
    P1_s_raw = _matrix_root_via_eig(P10_s_full, power=10)
    P1_s = _project_row_stochastic(P1_s_raw)
    P1_s_df = pd.DataFrame(P1_s, index=STATES, columns=STATES)
    out_name = "grade_transition_1y_pe.csv" if strat == "Private Equity" else "grade_transition_1y_vc.csv"
    P1_s_df.to_csv(os.path.join(DATA_DIR, out_name))

print(' Implied 10y matrix (A/B/C -> A/B/C/D):') 
display(P10_implied_df.loc[GRADE_FROM, GRADE_TO]) 

Eligible funds (>=40 quarters): 317
Transitions after filters: 258


,FundID,AdjStrategy,From,To
0,0084126E5AB84A059EFCD1CB88947783,Venture Capital,C,B
1,011713FB4D3441B2B00B36C4597B8287,Venture Capital,A,B
2,0228D6615AF74A869D41026B154A427A,Private Equity,B,A
6,05E5DE8506E245CBA5759579109FD95E,Venture Capital,A,A
7,0642339AE0EF498EBD8A4D2A3FA886D0,Venture Capital,B,D
8,0714283B8D534636BCABD1AE416BA365,Venture Capital,B,C
9,074FF0B76F664BF5984F26D5D8104C6E,Private Equity,A,B
12,084ACAB8A86449BE9774F4CAF72D5DF2,Venture Capital,C,C
13,08C044B11C4C4419BDCEF605D7A59051,Venture Capital,A,B
14,09797E2E0A714F1B95825D729A0B9D20,Private Equity,B,A


 COMBINED (VC + PE) Q1->Q40 COUNTS (From=A/B/C):


To,A,B,C,D
From,,,,
A,30,31,20,6
B,36,43,33,17
C,7,15,11,9


 COMBINED (VC + PE) Q1->Q40 PROBABILITIES (From=A/B/C):


...

To,A,B,C,D
From,,,,
A,0.404762,0.309524,0.261905,0.023810
B,0.253731,0.388060,0.208955,0.149254
C,0.187500,0.343750,0.187500,0.281250


Max abs error vs observed 10y (after stochastic projection): 0.02590678
 Derived 1y transition matrix (A/B/C/D):


,A,B,C,D
A,0.843254,0.108107,0.048639,0.000000
B,0.105916,0.787487,0.093339,0.013257
C,0.002698,0.159646,0.803427,0.034228
D,0.000000,0.000000,0.000000,1.000000


 Implied 10y matrix (A/B/C -> A/B/C/D):


,A,B,C,D
A,0.342472,0.345435,0.230632,0.081462
B,0.263389,0.355834,0.244481,0.136296
C,0.179596,0.331236,0.273129,0.216039
